# Sentiment Analysis on Amazon Food Reviews

## Importing Necessary Libraries

In [2]:
#importing Necessary libraries
import zipfile
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import os
from datetime import datetime
from glob import glob

import nltk
from textblob import TextBlob,Word
from nltk.corpus import stopwords
sw=stopwords.words("english")
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

from tqdm import tqdm,tqdm_notebook
tqdm.pandas()

import  warnings
warnings.filterwarnings("ignore")

# Unzipping and Checking the Format of the files

In [5]:
#Unzipping and Extracting the files
zip_doc=zipfile.ZipFile(r"C:\Users\hp\Desktop\inmtks\Projects\sent_text\txt_reviews.zip")
zip_doc.extractall(r"C:\Users\hp\Desktop\inmtks\Projects\sent_text\txt_reviews")
zip_doc.close()

In [2]:
zip_doc=zipfile.ZipFile(r"C:\Users\hp\Desktop\inmtks\Projects\sent_text\txt_reviews.zip")

In [3]:
files=os.listdir(r"C:\Users\hp\Desktop\inmtks\Projects\sent_text\txt_reviews")
print("No. of files:",len(files))

No. of files: 568455


In [4]:
files[0]

'review_1.txt'

In [6]:
#Checking the Format of the data in Text file
file_1=open(r"C:\Users\hp\Desktop\inmtks\Projects\sent_text\txt_reviews\\review_1.txt","r")
lines_1=file_1.readlines()
print(type(lines_1[0]))
lines_1

<class 'str'>


['ProductId: B001E4KFG0\n',
 'UserId: A3SGXH7AUHU8GW\n',
 'ProfileName: delmartian\n',
 'HelpfulnessNumerator: 1\n',
 'HelpfulnessDenominator: 1\n',
 'Score: 5\n',
 'Time: 1303862400\n',
 'ReviewSummary: Good Quality Dog Food\n',
 'ReviewText: I have bought several of the Vitality canned dog food products and have found them all to be of good quality. The product looks more like a stew than a processed meat and it smells better. My Labrador is finicky and she appreciates this product better than  most.\n']

#  Segregating and Appending the respective data into Lists

In [13]:
#Segregating the Data in appropraite way
product_id=[]
user_id=[]
p_name=[]
help_num=[]
help_den=[]
score=[]
time=[]
r_summary=[]
r_text=[]

for file in zip_doc.namelist():
    file=open(r"C:\Users\hp\Desktop\inmtks\Projects\sent_text\txt_reviews\\"+file,"r",encoding="utf-8")
    lines=file.readlines()
    product_id.append(lines[0].split(":")[1].strip("\n"))
    user_id.append(lines[1].split(":")[1].strip("\n"))
    p_name.append(lines[2].split(":")[1].strip("\n"))
    help_num.append(lines[3].split(":")[1].strip("\n"))
    help_den.append(lines[4].split(":")[1].strip("\n"))
    score.append(lines[5].split(":")[1].strip("\n"))
    time.append(lines[6].split(":")[1].strip("\n"))
    r_summary.append(lines[7].split(":")[1].strip("\n"))
    r_text.append(lines[8].split(":")[1].strip("\n"))
    
    

In [14]:
print(len(product_id),len(user_id),len(p_name),len(help_num),len(help_den),len(score),len(time),len(r_summary),
      len(r_text),sep="\n")

568454
568454
568454
568454
568454
568454
568454
568454
568454


# Creating DataFrame

In [22]:
#Creating the Data frame
df=pd.DataFrame({"ProductId":product_id,
                 "UserId":user_id,
                 "ProfileName":p_name,
                 "HelpfulnessNumerator":help_num,
                 "HelpfulnessDenominator":help_den,
                 "Score":score,
                 "Time":time,
                 "ReviewSummary":r_summary,
                 "ReviewText":r_text})
df

,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,ReviewSummary,ReviewText
0,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned ...
1,B00171APVA,A21BT40VZCCYT4,Carol A. Reed,0,0,5,1351209600,Healthy Dog Food,This is a very healthy dog food. Good for the...
2,B0019CW0HE,A2P6ACFZ8FTNVV,Melissa Benjamin,0,1,1,1331164800,Bad,I fed this to my Golden Retriever and he hate...
3,B006F2NYI2,A132DJVI37RB4X,Scottdrum,2,5,2,1332374400,"Not hot, not habanero","I have to admit, I was a sucker for the large..."
4,B000P41A28,A82WIMR4RSVLI,Emrose mom,0,1,4,1337472000,The best weve tried so far,We have a 7 week old... He had gas and consti...
...,...,...,...,...,...,...,...,...,...
568449,B000LQORDE,AL22WN8RBBOW7,"LifeInTheCity ""trogg""",2,5,5,1256428800,Good Stuff,I love these noodle. A little to spicy for t...
568450,B000LQORDE,A2P7HIRYYWVOBD,Mason,2,5,5,1254096000,yummy!,I just love it and will buy another box when ...
568451,B000LQORDE,A1K0ZH5MQFBA77,jennilight,2,5,4,1250985600,Tastes like More!,My late father in law used to have a rating s...
568452,B000LQORDE,A29FRN2O7LWINL,T. Tsai,2,5,5,1237766400,Great ramen,This is my favorite brand of Korean ramen. It...


# Saving The DataFrame

In [23]:
#saving the DataFrame in to .CSV file
df.to_csv(r"C:\Users\hp\Desktop\inmtks\Projects\sent_text\review.csv",index=False)

In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 568454 entries, 0 to 568453
Data columns (total 9 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   ProductId               568454 non-null  object
 1   UserId                  568454 non-null  object
 2   ProfileName             568454 non-null  object
 3   HelpfulnessNumerator    568454 non-null  object
 4   HelpfulnessDenominator  568454 non-null  object
 5   Score                   568454 non-null  object
 6   Time                    568454 non-null  object
 7   ReviewSummary           568454 non-null  object
 8   ReviewText              568454 non-null  object
dtypes: object(9)
memory usage: 39.0+ MB


# Data Cleaning

In [27]:
df.describe()

,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,ReviewSummary,ReviewText
count,568454,568454,568454,568454,568454,568454,568454,568454,568454
unique,74258,256059,218402,231,234,5,3168,295249,391202
top,B007JFMH8M,A3OXHLG6DIBRW8,"C. F. Hill ""CFH""",0,0,5,1350345600,Delicious!,"<a href=""http"
freq,913,448,451,303826,270052,363122,1143,2462,1446


In [28]:
#checking the Duplicates
df.duplicated().sum()

287

In [29]:
df.drop_duplicates(inplace=True)
df.duplicated().sum()

0

In [30]:
#Dealing with Null values
df.isnull().sum()

ProductId                 0
UserId                    0
ProfileName               0
HelpfulnessNumerator      0
HelpfulnessDenominator    0
Score                     0
Time                      0
ReviewSummary             0
ReviewText                0
dtype: int64

In [44]:
#Dealing with columns and respective datatypes
df["HelpfulnessNumerator"]=df["HelpfulnessNumerator"].astype("int64")
df["HelpfulnessDenominator"]=df["HelpfulnessDenominator"].astype("int64")
df["Score"]=df["Score"].astype("int64")
df["Time"]=df["Time"].astype("int64")

In [ ]:
df["Time"]=df["Time"].apply(lambda x:" ".join(re.findall("(.*)T",datetime.fromtimestamp(x).isoformat())))

In [55]:
df["Time"]=pd.to_datetime(df["Time"])

In [46]:
#Creating new columns from existing columns
df["Helpfulness"]=((df.HelpfulnessNumerator/df.HelpfulnessDenominator)*100).astype("float")

In [48]:
df["Helpfulness"]=df.Helpfulness.fillna(0)

In [49]:
df.head()

,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,ReviewSummary,ReviewText,Helpfulness
0,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400000000000,Good Quality Dog Food,I have bought several of the Vitality canned ...,100.0
1,B00171APVA,A21BT40VZCCYT4,Carol A. Reed,0,0,5,1351209600000000000,Healthy Dog Food,This is a very healthy dog food. Good for the...,0.0
2,B0019CW0HE,A2P6ACFZ8FTNVV,Melissa Benjamin,0,1,1,1331164800000000000,Bad,I fed this to my Golden Retriever and he hate...,0.0
3,B006F2NYI2,A132DJVI37RB4X,Scottdrum,2,5,2,1332374400000000000,"Not hot, not habanero","I have to admit, I was a sucker for the large...",40.0
4,B000P41A28,A82WIMR4RSVLI,Emrose mom,0,1,4,1337472000000000000,The best weve tried so far,We have a 7 week old... He had gas and consti...,0.0


In [56]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 568167 entries, 0 to 568453
Data columns (total 10 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   ProductId               568167 non-null  object        
 1   UserId                  568167 non-null  object        
 2   ProfileName             568167 non-null  object        
 3   HelpfulnessNumerator    568167 non-null  int64         
 4   HelpfulnessDenominator  568167 non-null  int64         
 5   Score                   568167 non-null  int64         
 6   Time                    568167 non-null  datetime64[ns]
 7   ReviewSummary           568167 non-null  object        
 8   ReviewText              568167 non-null  object        
 9   Helpfulness             568167 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(3), object(5)
memory usage: 47.7+ MB


In [58]:
df.describe()

,HelpfulnessNumerator,HelpfulnessDenominator,Score,Helpfulness
count,568167.000000,568167.000000,568167.000000,568167.000000
mean,1.744225,2.229264,4.183208,40.797915
std,7.638188,8.291425,1.310391,46.208277
min,0.000000,0.000000,1.000000,0.000000
25%,0.000000,0.000000,4.000000,0.000000
50%,0.000000,1.000000,5.000000,0.000000
75%,2.000000,2.000000,5.000000,100.000000
max,866.000000,923.000000,5.000000,300.000000


In [57]:
#saving the Cleaned DataFrame into .CSV file
df.to_csv(r"C:\Users\hp\Desktop\inmtks\Projects\sent_text\review_cleaned.csv",index=False)

# Data Preprocessing

In [101]:
#Creating the objects
stemmer=PorterStemmer()
lemma=WordNetLemmatizer()

In [89]:
#Defining cle the Data for NLP process
def clean_text(doc):
    """
    Cleans the document
    """
    #converting the data in lower case format
    doc = doc.lower()
    # joining the words and removing the Stopwords
    doc = " ".join([word for word in doc.split() if word not in sw])
    #removing unnecessary characters in text
    doc = re.sub(r'[^A-z0-9\s]','',doc)
    doc = re.sub(r'_','',doc)
    #checking with stopwords
    doc = [word for word in doc.split() if word not in sw]
    #checking with word corrections and sigularising the words
    doc = [Word(x).correct() for x in doc]
    doc = [Word(x).singularize() for x in doc]
    return " ".join(doc)

In [ ]:
# Defining the lematization process and Parts of speech
def do_lemma(doc):
    doc_list = nltk.pos_tag(doc.split())
    r_pos = 'n'
    output = []
    for word,pos in doc_list:
        if pos.startswith('V'):
            r_pos = 'v'
        elif pos.startswith('R'):
            r_pos = 'r'
        elif pos.startswith('J'):
            r_pos = 'a'
        output.append(lemma.lemmatize(word, pos=r_pos))
    return " ".join(output)

In [90]:
#cleaning the "review summary"
df["Clean_summary"]=df["ReviewSummary"].progress_apply(clean_text)

100%|███████████████████████████████████████████████████████████████████████| 568167/568167 [00:14<00:00, 39675.73it/s]


In [91]:
#cleaning the "review text" 
df["Clean_text"]=df["ReviewText"].progress_apply(clean_text)

100%|████████████████████████████████████████████████████████████████████████| 568167/568167 [02:43<00:00, 3479.01it/s]


In [97]:
#lemmatizing the "Clean summary" 
df["summary_lemma"]=df["Clean_summary"].progress_apply(do_lemma)

100%|████████████████████████████████████████████████████████████████████████| 568167/568167 [07:53<00:00, 1198.84it/s]


In [98]:
# Lemmatrzing the "Clean_text"
df["text_lemma"]=df["Clean_text"].progress_apply(do_lemma)

100%|█████████████████████████████████████████████████████████████████████████| 568167/568167 [39:06<00:00, 242.14it/s]


In [102]:
# Stemming the "Clean_summary"
df["summary_stem"]=df["Clean_summary"].progress_apply(lambda x :" ".join([stemmer.stem(i) for i in x.split()]))

100%|███████████████████████████████████████████████████████████████████████| 568167/568167 [00:46<00:00, 12166.58it/s]


In [103]:
# Stemming the "Clean_text"
df["text_stem"]=df["Clean_text"].progress_apply(lambda x :" ".join([stemmer.stem(i) for i in x.split()]))

100%|█████████████████████████████████████████████████████████████████████████| 568167/568167 [09:36<00:00, 986.14it/s]


In [ ]:
# Creating Data frame from Existing Dataframe with Need columns
df_1=df[["Clean_summary","Clean_text","summary_lemma","text_lemma","summary_stem","text_stem","Score"]]
df_1.head()

In [5]:
#Saving the DataFrame
df_1.to_csv(r"C:\Users\hp\Desktop\inmtks\Projects\sent_text\review_prepro.csv",index=False)

In [3]:
df_1=pd.read_csv(r"C:\Users\hp\Desktop\inmtks\Projects\sent_text\review_prepro.csv")
df_1

,Clean_summary,Clean_text,summary_lemma,text_lemma,summary_stem,text_stem,Score
0,good quality dog food,bought several vitality canned dog food produc...,good quality dog food,buy several vitality can dog food products fin...,good qualiti dog food,bought sever vital can dog food product found ...,5
1,healthy dog food,healthy dog food good digestion also good smal...,healthy dog food,healthy dog food good digestion also good smal...,healthi dog food,healthi dog food good digest also good small p...,5
2,bad,fed golden retriever hated eat gave terrible d...,bad,feed golden retriever hat eat give terrible di...,bad,fed golden retriev hate eat gave terribl diarr...,1
3,hot habanero,admit sucker large quantity 12 oz shopping hot...,hot habanero,admit sucker large quantity 12 oz shopping hot...,hot habanero,admit sucker larg quantiti 12 oz shop hot sauc...,2
4,best weve tried far,7 week old gas constipation problems first 5 w...,best weve try far,7 week old gas constipation problems first 5 w...,best weve tri far,7 week old ga constip problem first 5 week tri...,4
...,...,...,...,...,...,...,...
568162,good stuff,love noodle little spicy wife perfct,good stuff,love noodle little spicy wife perfct,good stuff,love noodl littl spici wife perfct,5
568163,yummy,love buy another box im done last one,yummy,love buy another box im do last one,yummi,love buy anoth box im done last one,5
568164,tastes like,late father law used rating system meals,taste like,late father law use rat system meals,tast like,late father law use rate system meal,4
568165,great ramen,favorite brand korean ramen spicy used eating ...,great ramen,favorite brand korean ramen spicy use eat spic...,great ramen,favorit brand korean ramen spici use eat spici...,5


In [4]:
df_1.columns

Index(['Clean_summary', 'Clean_text', 'summary_lemma', 'text_lemma',
       'summary_stem', 'text_stem', 'Score'],
      dtype='object')

In [5]:
# Separating input and output varaibles
x=df_1[["summary_lemma","text_lemma"]].fillna("")
y=df_1["Score"]

In [7]:
#Splitting the Train and Test data
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(x,y,train_size=0.7,random_state=45)

#checking with shape 
print(X_train.shape,X_test.shape)
print(Y_train.shape,Y_test.shape)

(397716, 2) (170451, 2)
(397716,) (170451,)


# Performing Bag words

In [13]:
#creating Bag of words 
from sklearn.feature_extraction.text import CountVectorizer
bow_l=CountVectorizer()
bow_s=CountVectorizer()

In [14]:
#applying the Bag words
x_train_bow_l_summary=bow_l.fit_transform(X_train["summary_lemma"])
x_train_bow_s_text=bow_s.fit_transform(X_train["text_lemma"])

In [15]:
#Checking with Total Number of Unique Words 
print("Total unique words:", len(bow_l.vocabulary_))
print("Total unique words:", len(bow_s.vocabulary_))

Total unique words: 33994
Total unique words: 168382


In [16]:
x_test_bow_l_summary=bow_l.transform(X_test["summary_lemma"])
x_test_bow_s_text=bow_s.transform(X_test["text_lemma"])

In [17]:
from scipy.sparse import hstack

In [18]:
x_train_trans_1=hstack((x_train_bow_l_summary,x_train_bow_s_text))# stacking the x_train summary and text columns 
x_test_trans_1=hstack((x_test_bow_l_summary,x_test_bow_s_text))# stacking the x_test summary and text columns 

# Model Building 

## Logistic Regression

In [22]:
#logistic Regression
from sklearn.linear_model import LogisticRegression
log=LogisticRegression()

#Training the model
log.fit(x_train_trans_1,Y_train)

#Preidict the model
y_test_pred=log.predict(x_test_trans_1)

#Evaluating the Model
from sklearn import metrics
lg_score=metrics.accuracy_score(Y_test,y_test_pred)


print("Accurracy:",lg_score*100)
print("Classification Report:\n",metrics.classification_report(Y_test,y_test_pred))

Accurracy: 76.93178684783311
Classification Report:
               precision    recall  f1-score   support

           1       0.70      0.73      0.71     15569
           2       0.51      0.34      0.41      9065
           3       0.54      0.42      0.47     12935
           4       0.56      0.35      0.43     24279
           5       0.83      0.95      0.89    108603

    accuracy                           0.77    170451
   macro avg       0.63      0.56      0.58    170451
weighted avg       0.74      0.77      0.75    170451



## Gradient Boosting 

In [27]:
#Gradient Boost Classifier
from sklearn.ensemble import GradientBoostingClassifier
gb=GradientBoostingClassifier(learning_rate=0.2)

#Training the Model
gb.fit(x_train_trans_1,Y_train)

#Predicting the Model
y_gb_pred=gb.predict(x_test_trans_1)

#Evaluating the Model
from sklearn import metrics
gbc_score=metrics.accuracy_score(Y_test,y_gb_pred)

print("Accurracy:",gbc_score*100)
print("Classification Report:\n",metrics.classification_report(Y_test,y_gb_pred))

Accurracy: 70.89310124317252
Classification Report:
               precision    recall  f1-score   support

           1       0.75      0.41      0.53     15569
           2       0.63      0.12      0.20      9065
           3       0.63      0.19      0.29     12935
           4       0.53      0.18      0.27     24279
           5       0.72      0.98      0.83    108603

    accuracy                           0.71    170451
   macro avg       0.65      0.38      0.43    170451
weighted avg       0.68      0.71      0.65    170451



## AdaBoosting 

In [23]:
#Adboost classifier
from sklearn.ensemble import AdaBoostClassifier
abg=AdaBoostClassifier(learning_rate=0.2)

#Training the Model
abg.fit(x_train_trans_1,Y_train)

#Predicting the Model
y_adb_pred=abg.predict(x_test_trans_1)

#Evaluating the Model
from sklearn import metrics
abg_score=metrics.accuracy_score(Y_test,y_adb_pred)

print("Accurracy:",abg_score*100)
print("Classification Report:\n",metrics.classification_report(Y_test,y_adb_pred))

Accurracy: 64.09583986013575
Classification Report:
               precision    recall  f1-score   support

           1       0.86      0.02      0.04     15569
           2       0.00      0.00      0.00      9065
           3       0.68      0.03      0.06     12935
           4       0.50      0.00      0.00     24279
           5       0.64      1.00      0.78    108603

    accuracy                           0.64    170451
   macro avg       0.53      0.21      0.17    170451
weighted avg       0.61      0.64      0.51    170451



## Decision Tree

In [24]:
#DecisionTreeClassifier
from sklearn.tree import DecisionTreeClassifier

dt=DecisionTreeClassifier(max_depth=2)

#Training the Model
dt.fit(x_train_trans_1, Y_train)

#Predict the Model
y_pred=dt.predict(x_test_trans_1)

#Evaluating the Model
from sklearn import metrics
dt_score=metrics.accuracy_score(Y_test,y_pred)

print("Accuracy:",dt_score*100)
print("Classification Report:\n",metrics.classification_report(Y_test,y_pred))

Accuracy: 63.77785991281952
Classification Report:
               precision    recall  f1-score   support

           1       0.00      0.00      0.00     15569
           2       0.00      0.00      0.00      9065
           3       0.00      0.00      0.00     12935
           4       0.42      0.02      0.04     24279
           5       0.64      1.00      0.78    108603

    accuracy                           0.64    170451
   macro avg       0.21      0.20      0.16    170451
weighted avg       0.47      0.64      0.50    170451



## Naive Bayes

In [25]:
#MultiNomialNaiveBayes
from sklearn.naive_bayes import MultinomialNB
mnb=MultinomialNB()

#Training The Model
mnb.fit(x_train_trans_1,Y_train)

#Predicting the Model
y_mnb_pred=mnb.predict(x_test_trans_1)

#Evaluating the Model
from sklearn import metrics
gnb_score=metrics.accuracy_score(Y_test,y_mnb_pred)

print("Accuracy:",gnb_score*100)
print("Classification Report:\n",metrics.classification_report(Y_test,y_mnb_pred))

Accuracy: 74.14036878633742
Classification Report:
               precision    recall  f1-score   support

           1       0.64      0.70      0.67     15569
           2       0.63      0.18      0.28      9065
           3       0.53      0.32      0.40     12935
           4       0.46      0.40      0.43     24279
           5       0.82      0.92      0.87    108603

    accuracy                           0.74    170451
   macro avg       0.61      0.51      0.53    170451
weighted avg       0.72      0.74      0.72    170451



In [41]:
#calculating the F1-score
lr_f1=metrics.f1_score(Y_test,y_test_pred,pos_label="positive",average="weighted")
mnb_f1=metrics.f1_score(Y_test,y_mnb_pred,pos_label="positive",average="weighted")
gb_f1=metrics.f1_score(Y_test,y_gb_pred,pos_label="positive",average="weighted")
dt_f1=metrics.f1_score(Y_test,y_pred,pos_label="positive",average="weighted")
ad_f1=metrics.f1_score(Y_test,y_adb_pred,pos_label="positive",average="weighted")

#calculating the Precision-score
lr_pre=metrics.precision_score(Y_test,y_test_pred,pos_label="positive",average="macro")
mnb_pre=metrics.precision_score(Y_test,y_mnb_pred,pos_label="positive",average="macro")
gb_pre=metrics.precision_score(Y_test,y_gb_pred,pos_label="positive",average="macro")
dt_pre=metrics.precision_score(Y_test,y_pred,pos_label="positive",average="macro")
ad_pre=metrics.precision_score(Y_test,y_adb_pred,pos_label="positive",average="macro")

#creating Lists for respective Metrics
acc=[lg_score*100,gbc_score*100,abg_score*100,dt_score*100,gnb_score*100]
f1=[lr_f1*100,gb_f1*100,ad_f1*100,dt_f1*100,mnb_f1*100]
prec=[lr_pre*100,gb_pre*100,ad_pre*100,dt_pre*100,mnb_pre*100]
mod=["Logistic Regression","Gradient Boosting","AdaBoosting","Decision Tree","Naive Bayes"]
#creating Dataframe for Comparision of Models

com_df=pd.DataFrame({"Model":mod,
                    "Accuracy":acc,
                    "Precision":prec,
                    "F1_Score":f1})
com_df

,Model,Accuracy,Precision,F1_Score
0,Logistic Regression,76.931787,63.011013,74.911996
1,Gradient Boosting,70.893101,65.154196,64.908922
2,AdaBoosting,64.095840,53.489928,50.501167
3,Decision Tree,63.777860,21.260172,50.144151
4,Naive Bayes,74.140369,61.432285,72.072159


# Conclusion:
1. The results of the evaluation using F1 score as the metric shows that , **Logistic Regression** Performing well compared to other models, with a score of **74.91**%.

2. Naive Bayes(Multinomial) and Gradient Bossint Classifier is the second-best performance, with a score of 72.07% and 64.91%

These results suggest that **Logistic Regression, Naive Bayes and Gradient Boosting** these three may be the most effective model for this particular task.

Although further evaluation and validation is necessary to confirm this conclusion.

**Logistic Regression, Naive Bayes**  we can use these two models for the deployment due time, speed of the execution and **F1 score**.

# Saving the Model

In [ ]:
#Saving the Model using Pickle
import pickle
pickle.dump(log,open(r"C:\Users\hp\Desktop\inmtks\Projects\sent_text\final_model_sent","wb"))